#### ingest the nested drivers json file

- read json file
- rename columns and add new columns
- drop unwanted columns
- write output to silver container

In [0]:
dbutils.widgets.text("data_source","testing")
value_data_source = dbutils.widgets.get("data_source")

In [0]:
%run "../../constants/configuration"


In [0]:
%run "../../utils/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp, col, concat, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [0]:
name_schema = StructType([
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
drivers_schema = StructType([
    StructField("driverId", IntegerType(), False),
    StructField("driverRef", StringType(), True),
    StructField("number", IntegerType(), True),
    StructField("code", StringType(), True),
    StructField("url", StringType(), True),
    StructField("name", name_schema),
    StructField("dob", DateType(), True),
    StructField("nationality", StringType(), True),
])

In [0]:
drivers_df = spark.read \
    .schema(drivers_schema)\
    .json(f"{bronze_container_path}/drivers.json")

In [0]:
drivers_with_ingestion_date_df = add_ingestion_date(drivers_df)

##### rename columns and add columns

In [0]:
drivers_with_columns_df = drivers_with_ingestion_date_df.withColumnRenamed("driverId", "driver_id")\
                                    .withColumnRenamed("driverRef", "driver_ref")\
                                    .withColumn("ingestion_date", current_timestamp())\
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))\
                                    .withColumn("data_source", lit(value_data_source))


##### drop column url

In [0]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

In [0]:
drivers_final_df.write.mode("overwrite").format("delta").saveAsTable("motor_dev.silver.drivers")